# FPProphet. Практика
В этом ноутбуке мы попытаемся прогнозировать данные по транзакциям с использованием библиотеки пророк

Загрузим библиотеки и данные (будем использовать для удобства оригинальный датасет с соревнования  Corporación Favorita Grocery Sales Forecastingg). Для этого придется эти данные разархивировать в рабочую директорию

In [ ]:

import os
!pip install pyunpack
!pip install patool
os.system('apt-get install p7zip')


from pyunpack import Archive
import shutil

In [ ]:

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        Archive(os.path.join(dirname, filename)).extractall('/kaggle/working/')
        print(os.path.join(dirname, filename))

In [ ]:
# Приведите данные train.csv к виду временного ряда, сгруппированного по дате, с агрегирующей функцией sum() для столбца unit_sales. 
# У вас должен получиться временной ряд с двумя столбцами: дата и sum_unit_sales. Отобразите на графике. 
# Видно ли влияние сезонности и других факторов на количество покупок?

In [ ]:
import numpy as np 
import pandas as pd 
# optimize memory loading by specifying the data types...
dtypes = {'id':'uint32', 'item_nbr':'int32', 'store_nbr':'int8', 'onpromotion':str}
train_df = pd.read_csv('./train.csv', dtype=dtypes, parse_dates=['date'])
# Если мы посмотрим на данные транзакций, транзакции сгруппированы по номерам магазинов. 
# Сейчас мы упростим это и сгруппируем их по дате. 
df = train_df.groupby("date")['unit_sales'].sum()
df.to_csv('./all_transactions.csv')

In [ ]:
# Load libraries
import numpy as np
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
import pandas as pd
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 70)
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode()

from fbprophet import Prophet

%time df_transactions = pd.read_csv('./all_transactions.csv')
%time df_holidays_events = pd.read_csv('./holidays_events.csv')

print('Data and libraries are loaded.')
df_transactions.head()

In [ ]:
df_transactions = df_transactions.set_index(pd.DatetimeIndex(df_transactions['date'])).drop(columns=['date'])
df_transactions.head()

In [ ]:
transactions = df_transactions['unit_sales']
py.iplot([go.Scatter(
    x=transactions.index,
    y=transactions
)])

Хорошо заметно влияние сезонности и праздников на общий объем транзакций.

Теперь давайте попробуем библиотеку пророк и посмотрим, насколько хорошо она предсказывает. Но перед этим мы должны подготовить данные. Согласно документации:

> Prophet следует API модели sklearn. Мы создаем экземпляр класса Prophet, а затем вызываем его методы соответствия и прогнозирования.
> Входными данными для Prophet всегда является фрейм данных с двумя столбцами: ** ds ** и ** y **. Столбец ds (отметка даты) должен содержать дату или дату и время (это нормально). Столбец ** y ** должен быть числовым и представлять измерение, которое мы хотим спрогнозировать.

In [ ]:
transactions = pd.DataFrame(transactions).reset_index()
transactions.columns = ['ds', 'y']
transactions

In [ ]:
# Попробуйте выполнить предсказание пророком (prophet) на год вперёд (periods=365). Отобразите на графике. Посчитайте RSME.

In [ ]:
m = Prophet()
m.fit(transactions)
future = m.make_future_dataframe(periods=365)
forecast = m.predict(future)
forecast

In [ ]:
py.iplot([
    go.Scatter(x=transactions['ds'], y=transactions['y'], name='y'),
    go.Scatter(x=forecast['ds'], y=forecast['yhat'], name='yhat'),
    go.Scatter(x=forecast['ds'], y=forecast['yhat_upper'], fill='tonexty', mode='none', name='upper'),
    go.Scatter(x=forecast['ds'], y=forecast['yhat_lower'], fill='tonexty', mode='none', name='lower'),
    go.Scatter(x=forecast['ds'], y=forecast['trend'], name='Trend')
])

In [ ]:
# Вычислим среднеквадратичную ошибку.
print('RMSE: %f' % np.sqrt(np.mean((forecast.loc[:1682, 'yhat']-transactions['y'])**2)) )

Как видно на графике выше, прогноз довольно хороший и согласуется с взлетами и падениями данных. Вы можете увеличить график, выбрав область масштабирования с помощью мыши.

Но тенденция довольно жесткая, она не учитывает под-тренды середины года. В первой половине года тенденция нарастает, а затем немного замедляется. Сделаем тренд немного гибким. Если тренд является переобученным (слишком большая гибкость) или недостаточными (недостаточная гибкость), вы можете отрегулировать силу разреженности перед использованием входного аргумента ** changepoint_prior_scale **. По умолчанию этот параметр установлен на 0,05. Его увеличение сделает тренд более гибким. (https://facebook.github.io/prophet/docs/trend_changepoints.html)

In [ ]:
# Поэкспериментируйте с коэффициентом changepoint_prior_scale (переберите 3-5 значений, больше/меньше 0,5) и посмотрите, как изменится тренд. 
# В какую сторону необходимо двигать коэффициент? Тренд переобучен или недообучен?

In [ ]:
m = Prophet(changepoint_prior_scale=0.1)
m.fit(transactions)
future = m.make_future_dataframe(periods=365)
forecast = m.predict(future)
# Вычислим среднеквадратичную ошибку.
print('RMSE: %f' % np.sqrt(np.mean((forecast.loc[:1682, 'yhat']-transactions['y'])**2)) )
py.iplot([
    go.Scatter(x=transactions['ds'], y=transactions['y'], name='y'),
    go.Scatter(x=forecast['ds'], y=forecast['yhat'], name='yhat'),
    go.Scatter(x=forecast['ds'], y=forecast['yhat_upper'], fill='tonexty', mode='none', name='upper'),
    go.Scatter(x=forecast['ds'], y=forecast['yhat_lower'], fill='tonexty', mode='none', name='lower'),
    go.Scatter(x=forecast['ds'], y=forecast['trend'], name='Trend')
])

In [ ]:
m = Prophet(changepoint_prior_scale=0.25)
m.fit(transactions)
future = m.make_future_dataframe(periods=365)
forecast = m.predict(future)
# Вычислим среднеквадратичную ошибку.
print('RMSE: %f' % np.sqrt(np.mean((forecast.loc[:1682, 'yhat']-transactions['y'])**2)) )
py.iplot([
    go.Scatter(x=transactions['ds'], y=transactions['y'], name='y'),
    go.Scatter(x=forecast['ds'], y=forecast['yhat'], name='yhat'),
    go.Scatter(x=forecast['ds'], y=forecast['yhat_upper'], fill='tonexty', mode='none', name='upper'),
    go.Scatter(x=forecast['ds'], y=forecast['yhat_lower'], fill='tonexty', mode='none', name='lower'),
    go.Scatter(x=forecast['ds'], y=forecast['trend'], name='Trend')
])

In [ ]:
m = Prophet(changepoint_prior_scale=0.5)
m.fit(transactions)
future = m.make_future_dataframe(periods=365)
forecast = m.predict(future)
# Вычислим среднеквадратичную ошибку.
print('RMSE: %f' % np.sqrt(np.mean((forecast.loc[:1682, 'yhat']-transactions['y'])**2)) )
py.iplot([
    go.Scatter(x=transactions['ds'], y=transactions['y'], name='y'),
    go.Scatter(x=forecast['ds'], y=forecast['yhat'], name='yhat'),
    go.Scatter(x=forecast['ds'], y=forecast['yhat_upper'], fill='tonexty', mode='none', name='upper'),
    go.Scatter(x=forecast['ds'], y=forecast['yhat_lower'], fill='tonexty', mode='none', name='lower'),
    go.Scatter(x=forecast['ds'], y=forecast['trend'], name='Trend')
])

In [ ]:
m = Prophet(changepoint_prior_scale=1)
m.fit(transactions)
future = m.make_future_dataframe(periods=365)
forecast = m.predict(future)
# Вычислим среднеквадратичную ошибку.
print('RMSE: %f' % np.sqrt(np.mean((forecast.loc[:1682, 'yhat']-transactions['y'])**2)) )
py.iplot([
    go.Scatter(x=transactions['ds'], y=transactions['y'], name='y'),
    go.Scatter(x=forecast['ds'], y=forecast['yhat'], name='yhat'),
    go.Scatter(x=forecast['ds'], y=forecast['yhat_upper'], fill='tonexty', mode='none', name='upper'),
    go.Scatter(x=forecast['ds'], y=forecast['yhat_lower'], fill='tonexty', mode='none', name='lower'),
    go.Scatter(x=forecast['ds'], y=forecast['trend'], name='Trend')
])

In [ ]:
m = Prophet(changepoint_prior_scale=1.5)
m.fit(transactions)
future = m.make_future_dataframe(periods=365)
forecast = m.predict(future)
# Вычислим среднеквадратичную ошибку.
print('RMSE: %f' % np.sqrt(np.mean((forecast.loc[:1682, 'yhat']-transactions['y'])**2)) )
py.iplot([
    go.Scatter(x=transactions['ds'], y=transactions['y'], name='y'),
    go.Scatter(x=forecast['ds'], y=forecast['yhat'], name='yhat'),
    go.Scatter(x=forecast['ds'], y=forecast['yhat_upper'], fill='tonexty', mode='none', name='upper'),
    go.Scatter(x=forecast['ds'], y=forecast['yhat_lower'], fill='tonexty', mode='none', name='lower'),
    go.Scatter(x=forecast['ds'], y=forecast['trend'], name='Trend')
])

In [ ]:
# Отобразите результаты с подобранным коэффициентом на графике. Как изменилась ошибка?

In [ ]:
m = Prophet(changepoint_prior_scale=2.5)
m.fit(transactions)
future = m.make_future_dataframe(periods=365)
forecast = m.predict(future)

In [ ]:
# Вычислим среднеквадратичную ошибку.
print('RMSE: %f' % np.sqrt(np.mean((forecast.loc[:1682, 'yhat']-transactions['y'])**2)) )
py.iplot([
    go.Scatter(x=transactions['ds'], y=transactions['y'], name='y'),
    go.Scatter(x=forecast['ds'], y=forecast['yhat'], name='yhat'),
    go.Scatter(x=forecast['ds'], y=forecast['yhat_upper'], fill='tonexty', mode='none', name='upper'),
    go.Scatter(x=forecast['ds'], y=forecast['yhat_lower'], fill='tonexty', mode='none', name='lower'),
    go.Scatter(x=forecast['ds'], y=forecast['trend'], name='Trend')
])

#### Проэкспериментировав с значением 'changepoint_prior_scale' in [0.1, 0.25, 0.5, 1, 1.5, 2.5]  можно отметить как с ростом значения 'changepoint_prior_scale' улучшалась "гибкость" тренда и уменьшалась недообученность модели. Ошибка RMSE уменьшилась с ростом значения 'changepoint_prior_scale'.

In [ ]:
# Добавьте в модель месячную сезонность (name='monthly', period=30.5, fourier_order=5). 
# Как изменилась ошибка? Улучшилась ли модель? Отобразите на графике. Посчитайте ошибку.

Теперь добавим в модель больше сезонности. Как мы видим, Пророк рассчитывает еженедельную и годовую сезонность. Нам не нужна ежедневная сезонность, потому что у нас нет внутридневных данных для этого случая. Достаточно просто добавить ежемесячную сезонность.

In [ ]:
m = Prophet(changepoint_prior_scale=2.5)
m.add_seasonality(name='monthly', period=30.5, fourier_order=5)
m.fit(transactions)
future = m.make_future_dataframe(periods=365)
forecast = m.predict(future)

In [ ]:
# Вычислим среднеквадратичную ошибку.
print('RMSE: %f' % np.sqrt(np.mean((forecast.loc[:1682, 'yhat']-transactions['y'])**2)) )
py.iplot([
    go.Scatter(x=transactions['ds'], y=transactions['y'], name='y'),
    go.Scatter(x=forecast['ds'], y=forecast['yhat'], name='yhat'),
    go.Scatter(x=forecast['ds'], y=forecast['yhat_upper'], fill='tonexty', mode='none', name='upper'),
    go.Scatter(x=forecast['ds'], y=forecast['yhat_lower'], fill='tonexty', mode='none', name='lower'),
    go.Scatter(x=forecast['ds'], y=forecast['trend'], name='Trend')
])

### Добавление сезонности в модель улучшило модель и существенно сократило ошибку!

In [ ]:
# Улучшите модель добавлением данных о праздниках. Отобразите результат, сделайте письменный вывод.

Пришло время добавить в нашу модель влияние праздников. Сначала нам нужно настроить формат данных. Пророку нужны два столбца (праздник и ds) и строка для каждого наступления праздника. Так же можно включить столбцы lower_window и upper_window, которые продлевают праздничные дни до дней «[lower_window, upper_window]» вокруг даты. 

In [ ]:
df_holidays_events

In [ ]:
holidays = df_holidays_events[df_holidays_events['transferred'] == False][['description', 'date']]
holidays.columns = ['holiday', 'ds']
#holidays['lower_window'] = 0
#holidays['upper_window'] = 0
holidays

In [ ]:
m = Prophet(changepoint_prior_scale=2.5, holidays=holidays)
m.add_seasonality(name='monthly', period=30.5, fourier_order=5)
m.fit(transactions)
future = m.make_future_dataframe(periods=365)
forecast = m.predict(future)

In [ ]:
# Вычислим среднеквадратичную ошибку.
print('RMSE: %f' % np.sqrt(np.mean((forecast.loc[:1682, 'yhat']-transactions['y'])**2)) )
py.iplot([
    go.Scatter(x=transactions['ds'], y=transactions['y'], name='y'),
    go.Scatter(x=forecast['ds'], y=forecast['yhat'], name='yhat'),
    go.Scatter(x=forecast['ds'], y=forecast['yhat_upper'], fill='tonexty', mode='none', name='upper'),
    go.Scatter(x=forecast['ds'], y=forecast['yhat_lower'], fill='tonexty', mode='none', name='lower'),
    go.Scatter(x=forecast['ds'], y=forecast['trend'], name='Trend')
])

Нам удалось спрогнозировать всплески на новогодний период. Модель не смогла уловить резкий скачок вниз на 4 января 2016 года, поэтому она не смогла успешно спрогнозировать 1 января 2017 года. 
Это объясняется тем, что 4 января 2016 года не было выходных. Но модель хорошо предсказывает продажи на 24 декабря. А также прогнозируемый период после 15 августа 2017 года выглядит неплохо.
### Добавление праздников в модель еще сильнее улучшило модель и сократило ошибку.